<a href="https://colab.research.google.com/github/mahanabba/flatiron_capstone/blob/main/CapstoneProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.2.1
!apt install openjdk-8-jdk-headless -qq
!pip install mlflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 60.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853643 sha256=77e3b88c2dc9ba721cef2fb6b7614b6e2b05f2a4a577fe50e17c6371d58e9044
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
openjdk-8-jdk-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.0 MB 59 kB/s 
     |████████████████████████████████| 147 kB 43.5 MB/s 
     |

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("truerev").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [ ]:
productgroup = df.groupby("product_title").count()

In [ ]:
reviews = productgroup.select("product_title", "count")
reviews.show()

+--------------------+-----+
|       product_title|count|
+--------------------+-----+
|Green Yarn Yoshi ...|   27|
|       GameShark Pro|   86|
|Shin Megami Tense...|   56|
|3DS/DSi/DSi XL Ca...|  140|
|Razer Carcharias ...|    3|
|PlayStation 3 - 3...|  491|
|               Joust|    9|
|Unreal Anthology ...|   74|
|RollerCoaster Tyc...|   95|
|3D Ultra Pinball:...|   12|
|Retro-Bit Genesis...|   51|
|Major League Base...|   17|
|Sega Dreamcast Sy...|    9|
|SEGA SATURN VIRTU...|    1|
|   AMAZING SPIDERMAN|    2|
|Neewer 2x Battery...|   58|
|Female USB to Ori...|    2|
|Full Parts Replac...|   16|
|Black and White (...|    6|
|Pearl Harbor-Zero...|    2|
+--------------------+-----+
only showing top 20 rows



In [ ]:
average_rating_df = df.groupby("product_title").agg({"star_rating":"avg", "product_title":"count"}).orderBy('count(product_title)', ascending=False)
average_rating_df.show()

+--------------------+------------------+--------------------+
|       product_title|  avg(star_rating)|count(product_title)|
+--------------------+------------------+--------------------+
|PlayStation 4 500...| 4.163787279220153|               10361|
|  Grand Theft Auto V| 4.544004589787722|                8715|
|Call of Duty: Ghosts|3.7910371318822023|                7810|
|       Battlefield 4|3.6683302141817427|                4809|
|  Assassin's Creed 4|4.5639559508682765|                4722|
|      The Last of Us| 4.733797303175294|                4598|
|Elder Scrolls V: ...| 4.307251487767247|                4537|
|             Destiny|3.7777777777777777|                4410|
| Diablo III - PC/Mac|2.7177676537585422|                4390|
|Call of Duty: Bla...| 4.065401326320604|                4373|
|SimCity - Limited...| 1.452920443101712|                3972|
|       Battlefield 3|3.8350619782443713|                3953|
|      Rocksmith 2014| 4.255569782330346|              

In [ ]:
vine_rating = df.groupby("vine").agg({"star_rating":"avg"})
vine_rating.show()

+----+-----------------+
|vine| avg(star_rating)|
+----+-----------------+
|   Y|4.074807737124213|
|   N|4.059856676690767|
+----+-----------------+



In [ ]:
numrows = df.count()
numrows

1785997

In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
nlp_df = df.select("star_rating", "review_body")
nlp_df.show()

+-----------+--------------------+
|star_rating|         review_body|
+-----------+--------------------+
|          5|Used this for Eli...|
|          5|Loved it,  I didn...|
|          1|poor quality work...|
|          3|nice, but tend to...|
|          4|Great amiibo, gre...|
|          1|The remote consta...|
|          5|I have a 2012-201...|
|          5|Perfect, kids lov...|
|          5|            Excelent|
|          4|Slippery but expe...|
|          5|Love the game. Se...|
|          1|Does not fit prop...|
|          2|This was way too ...|
|          4|Works great good ...|
|          1|It did not fit th...|
|          5|perfect lightweig...|
|          5|               great|
|          5|Works beautifully...|
|          5|Kids loved the ga...|
|          4|           Goodngame|
+-----------+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

In [ ]:
udf = UserDefinedFunction(lambda x: 'good' if x >= '3' else 'bad', StringType())

In [ ]:
nlpclass = nlp_df.withColumn('star_rating', udf('star_rating'))
nlpclass.show()


+-----------+--------------------+
|star_rating|         review_body|
+-----------+--------------------+
|       good|Used this for Eli...|
|       good|Loved it,  I didn...|
|        bad|poor quality work...|
|       good|nice, but tend to...|
|       good|Great amiibo, gre...|
|        bad|The remote consta...|
|       good|I have a 2012-201...|
|       good|Perfect, kids lov...|
|       good|            Excelent|
|       good|Slippery but expe...|
|       good|Love the game. Se...|
|        bad|Does not fit prop...|
|        bad|This was way too ...|
|       good|Works great good ...|
|        bad|It did not fit th...|
|       good|perfect lightweig...|
|       good|               great|
|       good|Works beautifully...|
|       good|Kids loved the ga...|
|       good|           Goodngame|
+-----------+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import length
nlpclass = nlpclass.withColumn('length', length(nlpclass['review_body']))
testdf = nlpclass.selectExpr("star_rating as class", "review_body as text", 'length as length')
testdf.count()

1785997

In [ ]:
testdfna = testdf.dropna(how='any')
testdfna.count()

1785893

In [ ]:
testdfna.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
| good|Used this for Eli...|   178|
| good|Loved it,  I didn...|   262|
|  bad|poor quality work...|    46|
| good|nice, but tend to...|    81|
| good|Great amiibo, gre...|    90|
|  bad|The remote consta...|    61|
| good|I have a 2012-201...|    85|
| good|Perfect, kids lov...|    64|
| good|            Excelent|     8|
| good|Slippery but expe...|    21|
| good|Love the game. Se...|    71|
|  bad|Does not fit prop...|   332|
|  bad|This was way too ...|   339|
| good|Works great good ...|    37|
|  bad|It did not fit th...|    95|
| good|perfect lightweig...|    52|
| good|               great|     5|
| good|Works beautifully...|    80|
| good|Kids loved the ga...|    40|
| good|           Goodngame|     9|
+-----+--------------------+------+
only showing top 20 rows



In [ ]:
 from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="token_text", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')

In [ ]:

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [ ]:
#build pipeline to transform data
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [ ]:
cleaner = data_prep_pipeline.fit(testdfna)
cleaned = cleaner.transform(testdfna)

In [ ]:
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[15775,19...|
|  0.0|(262145,[5537,656...|
|  1.0|(262145,[30950,34...|
|  0.0|(262145,[9129,275...|
|  0.0|(262145,[27576,29...|
|  1.0|(262145,[27576,95...|
|  0.0|(262145,[19036,49...|
|  0.0|(262145,[11382,14...|
|  0.0|(262145,[90499,26...|
|  0.0|(262145,[33917,12...|
|  0.0|(262145,[60392,62...|
|  1.0|(262145,[2437,722...|
|  1.0|(262145,[2437,592...|
|  0.0|(262145,[77073,11...|
|  1.0|(262145,[8540,190...|
|  0.0|(262145,[84677,11...|
|  0.0|(262145,[261870,2...|
|  0.0|(262145,[19036,20...|
|  0.0|(262145,[58267,74...|
|  0.0|(262145,[238383,2...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.classification import NaiveBayes
# split
training, testing = cleaned.randomSplit([0.7, 0.3])

#fit
nb = NaiveBayes()
predictor = nb.fit(training)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

test_results = predictor.transform(testing)

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.854323


In [ ]:
test_results.show(5)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|class|                text|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  bad|! week of playing...|   422|  1.0|[!, week, of, pla...|[!, week, playing...|(262144,[4991,107...|(262144,[4991,107...|(262145,[4991,107...|[-3049.9275061976...|[1.70420386382158...|       1.0|
|  bad|!!!DO NOT BUY!!!<...|   341|  1.0|[!!!do, not, buy!...|[!!!do, buy!!!<br...|(262144,[18176,19...|(262144,[18176,19...|(262145,[18176,19...|[-2399.5980889303...|[3.72348353599211...|       1.0|


In [ ]:
testpred = test_results.select("label", "prediction").toPandas()

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(testpred["label"], testpred["prediction"])
matrix

array([[385639,  65032],
       [ 19761,  66558]])

In [ ]:
#for saving and loading the model

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
model_save_name = 'classifier.json'
path = f"/content/gdrive/My Drive/{model_save_name}"
predictor.save(path)

In [ ]:
from pyspark.ml.classification import NaiveBayesModel

loaded = NaiveBayesModel.load(path)

In [ ]:
training2, testing2 = cleaned.randomSplit([0.999, 0.001])
loadedres = loaded.transform(testing2)
acc_eval2 = MulticlassClassificationEvaluator()
acc2 = acc_eval.evaluate(loadedres)
print(f'Accuracy of loaded model is {acc2}')

In [ ]:
#TODO = hyperparameter tuning with CV